# LLMChain with LCEL

In [32]:
#LLM Model
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

OPEN_AI_APIKEY="{YOUR_OPENAI_KEY}"
model = OpenAI(openai_api_key=OPEN_AI_APIKEY)

prompt = PromptTemplate.from_template("what is the famous tour place in {city}?")
chain = prompt | model
city = "Seoul"
chain.invoke({"city":city})
                                           

'\n\nOne of the most famous tour places in Seoul is the Gyeongbokgung Palace, which is the largest and most iconic royal palace in South Korea. It is known for its grand architecture, beautiful gardens, and rich history.'

# Sequential Chain with LCEL
ref : https://python.langchain.com/docs/expression_language/cookbook/multiple_chains

In [16]:
# Sequential Chain with LCEL
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

OPEN_AI_APIKEY="{YOUR_OPENAI_KEY}"
model = ChatOpenAI(openai_api_key=OPEN_AI_APIKEY)

prompt1 = PromptTemplate.from_template("what is the famous tour place in {city}? Tell me the name of the place only without additional comments.")
prompt2 = PromptTemplate.from_template("How can I get {place} by {transport}?")
chain1 = prompt1 | model 
chain2 = prompt2 | model 
chain = {"place":chain1,"transport":itemgetter("transport")} | chain2
output = chain.invoke({"city": "Seoul", "transport": "subway"})
print(output)

content='To get to Gyeongbokgung Palace in Seoul, South Korea by subway, you can follow these steps:\n\n1. Find the nearest subway station to your location.\n2. Check the subway map or use a navigation app to determine the best subway line and route to reach Gyeongbokgung Palace.\n3. Board the subway train heading towards the designated direction.\n4. Depending on your starting point, you may need to change subway lines or transfer at different stations.\n5. Keep an eye on the subway announcements and electronic displays to ensure you are on the right train and heading in the correct direction.\n6. Get off at the designated subway station for Gyeongbokgung Palace. The nearest station is "Gyeongbokgung Station" (Seoul Subway Line 3), Exit 5.\n7. Follow the signs or ask for directions to the palace. It is within walking distance from the subway station.\n8. Enjoy your visit to Gyeongbokgung Palace!\n\nNote: It is always recommended to check the latest subway schedules and any potential s

# Sequential Advacned with LCEL

In [25]:
from operator import itemgetter
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.schema import StrOutputParser

OPEN_AI_APIKEY="{YOUR_OPENAI_KEY}"
model = OpenAI(openai_api_key=OPEN_AI_APIKEY)

prompt1 = PromptTemplate.from_template("what is the famous tour place in {city}? Tell me the name of the place only without additional comments.")
prompt2 = PromptTemplate.from_template("What is the top 5 restaurant in the {place} in city {city} without additional comments?") #output : restaurants
prompt3 = PromptTemplate.from_template("What is the best one restaurant and food for family dinner among {restaurants} ?") #output : restaurant_information
prompt4 = PromptTemplate.from_template("How can I get the {place} by using {transport}?") #output : transport_information
final_prompt = PromptTemplate.from_template("""
Please summarize the tour information with reastaurant information and transportation by using the this information.
Restaurant informations : {restaurant_information}
Transport information : {transport_information}
""")
            
chain1 = {"city":itemgetter("city")} | prompt1 | model | StrOutputParser()
chain2 = {"place":chain1,"city":itemgetter("city")} | prompt2 | model | StrOutputParser()
chain3 = {"restaurants":chain2} | prompt3 | model |StrOutputParser()
chain4 = {"place":chain1,"transport":itemgetter("transport")} | prompt4 | model | StrOutputParser()
final_chain = { "restaurant_information":chain3 , "transport_information":chain4 } | final_prompt | model | StrOutputParser()
output = final_chain.invoke({"city": "Seoul", "transport": "subway"})

print(output)


The Gyeongbokgung Palace tour offers a great opportunity to experience traditional Korean culture. For a family dinner, Gogung Korean Royal Cuisine is a popular choice due to its welcoming atmosphere and diverse menu options. To get to the palace, you can take Line 3 or 5 of the subway and get off at Gyeongbokgung or Gwanghwamun Station, respectively. Both stations have underground tunnels that lead directly to the palace. 


# Router with LCEL


In [34]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from operator import itemgetter
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnableBranch

OPEN_AI_APIKEY="{YOUR_OPENAI_KEY}"
model = OpenAI(openai_api_key=OPEN_AI_APIKEY)

restaurant_template = """
You are a tourist guide. You know many good restaurants around the tourist destination.
You can recommend good foods and restaurants.
Here is a question:
{input}
"""

transport_template = """
You are a tourist guide. You have a lot of knowledge in public transportation.
You can provide information about public transportation to help tourists get to the tourist destination.
Here is a question:
{input}
"""

destination_template = """
You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
{input}
"""

prompt_infos = [
    {
        "name":"restaurants",
        "description":"Good for recommending restaurants around the tourist destinations",
        "prompt_template": restaurant_template
    },
    {
        "name":"transport",
        "description":"Good for guiding the transport to get the place",
        "prompt_template": transport_template
    },
    {
        "name":"destination",
        "description":"Good for recommending place to tour",
        "prompt_template": destination_template
    }
]

destination_chains = {}
for prompt_info in prompt_infos:
    name = prompt_info["name"]
    prompt = PromptTemplate.from_template(prompt_info["prompt_template"])
    chain = LLMChain(llm = model, prompt=prompt,verbose=True)
    destination_chains[name] = chain
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

default_prompt = PromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=model, prompt=default_prompt)

router_prompt = PromptTemplate(template="""
For the given question, refer to the explanations for the following categories below and select the appropriate category. 
If there is no suitable category, return the string "default".
The output should be single word 

Categories are composed of strings in the format "Category: Description", and the categories are as follows:
{destinations_str}

Question:
{input}
""",input_variables=["input"],partial_variables={"destinations_str":destinations_str})

print(destinations_str)
router_chain = {"input":itemgetter("input")} |router_prompt | model | StrOutputParser()

#branch = RunnableBranch(
#    (lambda x: "restaurants" in x['destination'].lower(), destination_chains['restaurants']),
#    (lambda x: "transport" in x['destination'].lower(), destination_chains['transport']),
#    (lambda x: "destination" in x['destination'].lower(), destination_chains['destination']),
#    default_chain,
#)

branch = RunnableBranch(
    *(
        (lambda x, name=prompt_info["name"], chain=destination_chains[name]: name in x['destination'].replace("\n", ""), chain) 
        for prompt_info in prompt_infos
    ),
    default_chain
)

full_chain = {"input":itemgetter("input"),"destination":router_chain} | branch 

input = "What is best restaurant in Seoul?"
output = full_chain.invoke({"input":input})
   
print(output['text'])

restaurants: Good for recommending restaurants around the tourist destinations
transport: Good for guiding the transport to get the place
destination: Good for recommending place to tour


> Entering new LLMChain chain...
Prompt after formatting:

You are a tourist guide. You know many good tourist places.
You can recommend good tourist places to the tourists.
Here is a question:
What is best restaurant in Seoul?


> Finished chain.

As a tourist guide, I highly recommend visiting the famous Gwangjang Market in Seoul. It is a bustling market filled with a variety of food stalls offering traditional Korean dishes such as bibimbap, tteokbokki, and mandu. Among the many options, I would say the best restaurant in Seoul is a small stall called "Myeongdong Kyoja" which specializes in handmade kalguksu (knife-cut noodles) and mandu (dumplings). The dishes are made with fresh and high-quality ingredients, and the flavors are simply delicious. The restaurant is also conveniently located near p